In [1]:
import matplotlib
matplotlib.use('Agg')
import os
import datetime
import numpy as np
import dill as pickle
import random
import sys
np.random.seed(0)
random.seed(0)
now = datetime.datetime.now().strftime("%B_%d_%Y_%H_%M_%S")
workingdirectory = os.popen('git rev-parse --show-toplevel').read()[:-1]
sys.path.append(workingdirectory)
os.chdir(workingdirectory)
#print(os.getcwd())
from codes.experimentclasses.SwissRoll49 import SwissRoll49
from codes.otherfunctions.multirun import get_coeffs_reps
from codes.otherfunctions.multirun import get_grads_reps_noshape_tangent
from codes.otherfunctions.multiplot import plot_betas_customcolors2
import copy
from codes.geometer.RiemannianManifold import RiemannianManifold
from codes.geometer.ShapeSpace import ShapeSpace
from codes.geometer.TangentBundle import TangentBundle

#set parameters
n = 100000 #number of data points to simulate
nsel = 100 #number of points to analyze with lasso
itermax = 1000 #maximum iterations per lasso run
tol = 1e-10 #convergence criteria for lasso
#lambdas = np.asarray([5,10,15,20,25,50,75,100], dtype = np.float16)#lambda values for lasso
#lambdas = np.asarray([0,2.95339658e-06, 5.90679317e-06, 8.86018975e-06, 1.18135863e-05,
#       1.47669829e-05, 2.95339658e-05, 4.43009487e-05, 5.90679317e-05])
#lambdas = np.asarray([0,.0001,.001,.01,.1,1,10], dtype = np.float64)
lambdas = np.asarray(np.hstack([np.asarray([0]),np.logspace(-3,1,11)]), dtype = np.float16)
n_neighbors = 1000 #number of neighbors in megaman
n_components = 2 #number of embedding dimensions (diffusion maps)
#10000 points and .17 dt is best so far
#100000 and .05 is better
#thats with 50 neighbors
#diffusion_time = 0.05 #diffusion time controls gaussian kernel radius per gradients paper
#diffusion_time = 0.1
#diffusion_time = 0.75#0.28#0.05
dim = 2 #manifold dimension
cores = 16 #number of cores for parallel processing

experiment = SwissRoll49(xvar = 0.1,cores = cores, noise = False)
experiment.M = experiment.generate_data(n = n,theta = np.pi/4) #if noise == False then noise parameters are overriden
experiment.q = n_components
radius = .1
def compute_nbr_wts_fast(A, sample, radius):
    Ps = list()
    nbrs = list()
    for ii in range(len(sample)):
        dists_sq = np.linalg.norm(experiment.M.data - experiment.M.data[ii], axis = 1)**2
        dists_sc = dists_sq / radius
        nbrs.append(np.where(dists_sc < 0.001)[0])
        dists_sc[np.where(dists_sc < 0.001)[0]] = 0
        w = np.exp(-dists_sc)
        #w = np.array(A[sample[ii],:].todense()).flatten()
        p = w / np.sum(w)
        #nbrs.append(np.where(p > 0)[0])
        Ps.append(p[nbrs[ii]])
    return(Ps, nbrs)

def get_wlpca_tangent_sel_fast(M, selectedpoints,  radius, dim = None):

    n = M.data.shape[0]
    nsel = len(selectedpoints)
    if dim == None:
        dim = self.dim
    data = M.data
    A = M.data
    (PS, nbrs) = compute_nbr_wts_fast(A, selectedpoints,radius)
    d = M.data.shape[1]
    tangent_bases = np.zeros((nsel, d, dim))
    for i in range(nsel):
        # print(i)
        p = PS[i]
        nbr = nbrs[i]
        Z = (data[nbr, :] - np.dot(p, data[nbr, :])) * p[:, np.newaxis]
        sig = np.dot(Z.transpose(), Z)
        e_vals, e_vecs = np.linalg.eigh(sig)
        j = e_vals.argsort()[::-1]  # Returns indices that will sort array from greatest to least.
        e_vec = e_vecs[:, j]
        e_vec = e_vec[:, :dim]
        tangent_bases[i, :, :] = e_vec
    return (tangent_bases)
def get_grads_reps_noshape_tangent_fast(experiment, nreps, nsel, cores,radius, dim):

    experiments = {}
    dim = experiment.dim
    for i in range(nreps):
        experiments[i] = copy.copy(experiment)
        experiments[i].M.selected_points = np.random.choice(list(range(experiment.n)), nsel, replace=False)
        experiments[i].selected_points = experiments[i].M.selected_points
        tangent_bases = get_wlpca_tangent_sel_fast(experiments[i].M, experiments[i].M.selected_points, radius)
        subM = RiemannianManifold(experiments[i].M.data[experiments[i].M.selected_points], dim)
        subM.tb = TangentBundle(subM, tangent_bases)
        experiments[i].df_M = np.asarray([np.identity(dim) for i in range(nsel)])
        experiments[i].dg_x = experiments[i].get_dx_g_full(experiments[i].M.data[experiments[i].M.selected_points])
        experiments[i].dg_x_norm = experiments[i].normalize(experiments[i].dg_x)
        experiments[i].dg_M = experiments[i].project(tangent_bases, experiments[i].dg_x_norm)
    return(experiments)

experiment.M.selected_points = np.random.choice(list(range(n)),nsel,replace = False)
experiment.selected_points = experiment.M.selected_points
nreps = 5
#import pickle
#with open('ethanolsavegeom1.pkl', 'wb') as output:
#    pickle.dump(experiment.N, output, pickle.HIGHEST_PROTOCOL)
print('pregrad',datetime.datetime.now().strftime("%B_%d_%Y_%H_%M_%S"))
import matplotlib.pyplot as plt
#plt.scatter(experiment.N.data[:,0], experiment.N.data[:,1])
#folder = workingdirectory + '/Figures/swissroll/' + now
#os.mkdir(folder)
#experiment.N.plot([0,1],list(range(n)), experiment.ts,.25,1,folder + '/c1', False)
#experiment.N.plot([0,1],list(range(n)), experiment.ys,.25,1,folder + '/c0', False)
#plt.scatter(experiment.N.data[:,0], experiment.N.data[:,1], c = experiment.ts)
#plt.savefig('/Users/samsonkoelle/Desktop/swizz' + str(now))
experiments = get_grads_reps_noshape_tangent_fast(experiment, nreps, nsel,cores, radius, dim)


def get_betas_spam2(xs, ys, groups, lambdas, n, q, itermax, tol):
    # n = xs.shape[0]
    p = len(np.unique(groups))
    lambdas = np.asarray(lambdas, dtype=np.float64)
    yadd = np.expand_dims(ys, 1)
    groups = np.asarray(groups, dtype=np.int32) + 1
    W0 = np.zeros((xs.shape[1], yadd.shape[1]), dtype=np.float32)
    Xsam = np.asfortranarray(xs, dtype=np.float32)
    Ysam = np.asfortranarray(yadd, dtype=np.float32)
    coeffs = np.zeros((len(lambdas), q, n, p))
    for i in range(len(lambdas)):
        # alpha = spams.fistaFlat(Xsam,Dsam2,alpha0sam,ind_groupsam,lambda1 = lambdas[i],mode = mode,itermax = itermax,tol = tol,numThreads = numThreads, regul = "group-lasso-l2")
        # spams.fistaFlat(Y,X,W0,TRUE,numThreads = 1,verbose = TRUE,lambda1 = 0.05, it0 = 10, max_it = 200,L0 = 0.1, tol = 1e-3, intercept = FALSE,pos = FALSE,compute_gram = TRUE, loss = 'square',regul = 'l1')
        output = spams.fistaFlat(Ysam, Xsam, W0, True, groups=groups, numThreads=-1, verbose=True,
                                     lambda1=lambdas[i], it0=100, max_it=itermax, L0=0.5, tol=tol, intercept=False,
                                     pos=False, compute_gram=True, loss='square', regul='group-lasso-l2', ista=False,
                                     subgrad=False, a=0.1, b=1000)
        coeffs[i, :, :, :] = np.reshape(output[0], (q, n, p))
        # print(output[1])
    return (coeffs)

def get_coeffs(experiment, lambdas, itermax, nsel, tol):
    experiment.xtrain, experiment.groups = experiment.construct_X(experiment.dg_M)
    experiment.ytrain = experiment.construct_Y(experiment.df_M, list(range(nsel)))
    experiment.coeffs = get_betas_spam2(experiment.xtrain, experiment.ytrain, experiment.groups, lambdas,
                                        nsel, experiment.dim, itermax, tol)
    return (experiment)


def get_coeffs_parallel(experiments, nreps, lambdas, itermax, nsel, tol, cores):
    p = Pool(cores)
    results = p.map(
        lambda i: get_coeffs(experiment=experiments[i], lambdas=lambdas, itermax=itermax, nsel=nsel, tol=tol),
        range(nreps))
    output = {}
    for i in range(nreps):
        output[i] = results[i]
    return (output)


from pathos.multiprocessing import ProcessingPool as Pool
import spams




savename = 'swissroll_020220_10p'
savefolder = 'swissroll'
experiments = get_coeffs_parallel(experiments, nreps, lambdas, itermax, nsel, tol, cores)

savename = 'swissroll_020520_samosa'
savefolder = 'swissroll'
#experiments = get_coeffs_parallel(experiments, nreps, lambdas, itermax, nsel, tol, cores)
with open(workingdirectory + '/untracked_data/embeddings/' + savefolder + '/' + savename + 's.pkl',
        'wb') as output:
    pickle.dump(experiments, output, pickle.HIGHEST_PROTOCOL)

xaxis = np.asarray(lambdas, dtype = np.float64) * np.sqrt(nsel * 2)
title ='Swiss Roll'
gnames = np.asarray(list(range(experiment.p)), dtype = str)
#folder = workingdirectory + '/Figures/swissroll/' + now
#os.mkdir(folder)
filename = folder + '/betas1p'
print('preplot',datetime.datetime.now().strftime("%B_%d_%Y_%H_%M_%S"))
colors = np.hstack([np.repeat('red',2), np.repeat('black',49)])
legtitle = 'Function type'
color_labels = np.asarray(['Manifold coordinates','Ambient coordinates'])
#plot_betas(experiments, xaxis, title,filename, gnames,nsel)
plot_betas_customcolors2(experiments, xaxis, title,filename, gnames,nsel,colors, legtitle, color_labels)


/Users/samsonkoelle/Downloads/manigrad-100818/mani-samk-gradients


In [8]:
#i = 0

#dists_sq = np.linalg.norm(experiment.M.data - experiment.M.data[i], axis = 1)**2
#cutoff = .3

In [10]:
#dist_to_nebs = experiment.M.data[np.where(dists_sq < cutoff**2)[0]] - experiment.M.data[i]

pregrad February_05_2020_16_44_46


In [ ]:

def get_betas_spam2(xs, ys, groups, lambdas, n, q, itermax, tol):
    # n = xs.shape[0]
    p = len(np.unique(groups))
    lambdas = np.asarray(lambdas, dtype=np.float64)
    yadd = np.expand_dims(ys, 1)
    groups = np.asarray(groups, dtype=np.int32) + 1
    W0 = np.zeros((xs.shape[1], yadd.shape[1]), dtype=np.float32)
    Xsam = np.asfortranarray(xs, dtype=np.float32)
    Ysam = np.asfortranarray(yadd, dtype=np.float32)
    coeffs = np.zeros((len(lambdas), q, n, p))
    for i in range(len(lambdas)):
        # alpha = spams.fistaFlat(Xsam,Dsam2,alpha0sam,ind_groupsam,lambda1 = lambdas[i],mode = mode,itermax = itermax,tol = tol,numThreads = numThreads, regul = "group-lasso-l2")
        # spams.fistaFlat(Y,X,W0,TRUE,numThreads = 1,verbose = TRUE,lambda1 = 0.05, it0 = 10, max_it = 200,L0 = 0.1, tol = 1e-3, intercept = FALSE,pos = FALSE,compute_gram = TRUE, loss = 'square',regul = 'l1')
        output = spams.fistaFlat(Ysam, Xsam, W0, True, groups=groups, numThreads=-1, verbose=True,
                                     lambda1=lambdas[i], it0=100, max_it=itermax, L0=0.5, tol=tol, intercept=False,
                                     pos=False, compute_gram=True, loss='square', regul='group-lasso-l2', ista=False,
                                     subgrad=False, a=0.1, b=1000)
        coeffs[i, :, :, :] = np.reshape(output[0], (q, n, p))
        # print(output[1])
    return (coeffs)

def get_coeffs(experiment, lambdas, itermax, nsel, tol):
    experiment.xtrain, experiment.groups = experiment.construct_X(experiment.dg_M)
    experiment.ytrain = experiment.construct_Y(experiment.df_M, list(range(nsel)))
    experiment.coeffs = get_betas_spam2(experiment.xtrain, experiment.ytrain, experiment.groups, lambdas,
                                        nsel, experiment.dim, itermax, tol)
    return (experiment)


def get_coeffs_parallel(experiments, nreps, lambdas, itermax, nsel, tol, cores):
    p = Pool(cores)
    results = p.map(
        lambda i: get_coeffs(experiment=experiments[i], lambdas=lambdas, itermax=itermax, nsel=nsel, tol=tol),
        range(nreps))
    output = {}
    for i in range(nreps):
        output[i] = results[i]
    return (output)


from pathos.multiprocessing import ProcessingPool as Pool
import spams




savename = 'swissroll_020220_10p'
savefolder = 'swissroll'
experiments = get_coeffs_parallel(experiments, nreps, lambdas, itermax, nsel, tol, cores)

In [48]:
savename = 'swissroll_020520_samosa'
savefolder = 'swissroll'
#experiments = get_coeffs_parallel(experiments, nreps, lambdas, itermax, nsel, tol, cores)
with open(workingdirectory + '/untracked_data/embeddings/' + savefolder + '/' + savename + 's.pkl',
        'wb') as output:
    pickle.dump(experiments, output, pickle.HIGHEST_PROTOCOL)

xaxis = np.asarray(lambdas, dtype = np.float64) * np.sqrt(nsel * 2)
title ='Swiss Roll'
gnames = np.asarray(list(range(experiment.p)), dtype = str)
#folder = workingdirectory + '/Figures/swissroll/' + now
#os.mkdir(folder)
filename = folder + '/betas1p'
print('preplot',datetime.datetime.now().strftime("%B_%d_%Y_%H_%M_%S"))
colors = np.hstack([np.repeat('red',2), np.repeat('black',49)])
legtitle = 'Function type'
color_labels = np.asarray(['Manifold coordinates','Ambient coordinates'])
#plot_betas(experiments, xaxis, title,filename, gnames,nsel)
plot_betas_customcolors2(experiments, xaxis, title,filename, gnames,nsel,colors, legtitle, color_labels)


preplot February_05_2020_14_54_48
[[249.19271634 279.65077055 253.504589    23.78742738   7.06910287
    0.           0.           0.           0.           0.
    0.           0.        ]
 [252.43792996 286.6104764  264.39683773  40.27601344   7.90600294
    0.           0.           0.           0.           0.
    0.           0.        ]
 [238.86648265 258.78620313 220.87378541   6.51076991   2.07202039
    0.           0.           0.           0.           0.
    0.           0.        ]
 [254.06340288 289.8971474  269.54948783  47.06637968   7.25893115
    0.           0.           0.           0.           0.
    0.           0.        ]
 [241.4968126  264.15206429 229.60875084   2.89910714   0.45364036
    0.           0.           0.           0.           0.
    0.           0.        ]]
[[ 430.16588381  660.09804906  876.9510203  1040.18061421 1023.48510649
   611.1061758     0.            0.            0.            0.
     0.            0.        ]
 [ 428.23684894  656.47

[[47.02421274 49.93081113 30.56710401 11.92103596  0.          0.
   0.          0.          0.          0.          0.          0.        ]
 [46.5690386  49.50316799 30.37946677 11.84456705  0.          0.
   0.          0.          0.          0.          0.          0.        ]
 [47.34824685 50.3428319  30.71675626 11.94048552  0.          0.
   0.          0.          0.          0.          0.          0.        ]
 [46.47232213 49.40008781 30.34132483 11.83880716  0.          0.
   0.          0.          0.          0.          0.          0.        ]
 [47.50851785 50.42537341 30.74598507 11.97339382  0.          0.
   0.          0.          0.          0.          0.          0.        ]]
[[17.41865494  9.20683164  0.72257939  0.62358072  0.          0.
   0.          0.          0.          0.          0.          0.        ]
 [17.17443321  9.12022704  0.7744684   0.55080314  0.          0.
   0.          0.          0.          0.          0.          0.        ]
 [17.4544855

[[60.40911065  3.76574942  1.51864927  0.          0.          0.
   0.          0.          0.          0.          0.          0.        ]
 [60.10562379  3.54410154  1.52833318  0.          0.          0.
   0.          0.          0.          0.          0.          0.        ]
 [61.136154    4.38554121  1.48062497  0.          0.          0.
   0.          0.          0.          0.          0.          0.        ]
 [59.97506637  3.4287759   1.5254607   0.          0.          0.
   0.          0.          0.          0.          0.          0.        ]
 [60.96061481  4.23469551  1.48422242  0.          0.          0.
   0.          0.          0.          0.          0.          0.        ]]
[[39.24183564  2.70163391  0.36803083  0.          0.          0.
   0.          0.          0.          0.          0.          0.        ]
 [39.0218418   2.69140934  0.36463479  0.          0.          0.
   0.          0.          0.          0.          0.          0.        ]
 [39.6874601

[[141.28510976  72.27643759  13.2970421    2.41491568   0.
    0.           0.           0.           0.           0.
    0.           0.        ]
 [140.72203217  71.49240338  13.42582102   2.40930596   0.
    0.           0.           0.           0.           0.
    0.           0.        ]
 [143.15278303  75.2503326   12.57169791   2.25887994   0.
    0.           0.           0.           0.           0.
    0.           0.        ]
 [140.41423402  71.04672939  13.50100481   2.416021     0.
    0.           0.           0.           0.           0.
    0.           0.        ]
 [142.52741467  74.1903727   12.90149085   2.35391674   0.
    0.           0.           0.           0.           0.
    0.           0.        ]]
[[118.66269414  31.92480836  10.17401891   2.52733568   0.
    0.           0.           0.           0.           0.
    0.           0.        ]
 [118.19985409  31.39905592   9.6741344    2.19345827   0.
    0.           0.           0.           0.           0.

[[114.10488744  75.02436637   6.38663078   1.04549516   0.
    0.           0.           0.           0.           0.
    0.           0.        ]
 [113.73149541  74.70711767   6.43506737   1.00550775   0.
    0.           0.           0.           0.           0.
    0.           0.        ]
 [115.69821717  76.78277042   6.87890656   1.01160636   0.
    0.           0.           0.           0.           0.
    0.           0.        ]
 [113.48186327  74.4474617    6.37851431   1.03291312   0.
    0.           0.           0.           0.           0.
    0.           0.        ]
 [115.09048314  76.0703655    6.59521527   1.05764325   0.
    0.           0.           0.           0.           0.
    0.           0.        ]]
[[109.42636431   9.65069551   5.30955889   0.           0.
    0.           0.           0.           0.           0.
    0.           0.        ]
 [108.94957416   9.74417683   5.13663587   0.           0.
    0.           0.           0.           0.           0.

[[194.11730021 117.01822567  13.46879777   4.30360754   0.
    0.           0.           0.           0.           0.
    0.           0.        ]
 [194.15364187 117.22422155  13.46008729   4.31523624   0.
    0.           0.           0.           0.           0.
    0.           0.        ]
 [194.10028723 116.7391037   13.46867568   4.2620431    0.
    0.           0.           0.           0.           0.
    0.           0.        ]
 [194.1622249  117.27247354  13.45991896   4.30268932   0.
    0.           0.           0.           0.           0.
    0.           0.        ]
 [194.08753106 116.72343121  13.47146652   4.218219     0.
    0.           0.           0.           0.           0.
    0.           0.        ]]
[[88.86904555  6.73814955  6.16899991  0.          0.          0.
   0.          0.          0.          0.          0.          0.        ]
 [88.88709496  6.77324372  6.16341456  0.          0.          0.
   0.          0.          0.          0.          0.    

[[15.80079289  0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.        ]
 [15.81167195  0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.        ]
 [15.80789036  0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.        ]
 [15.81261116  0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.        ]
 [15.79375199  0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.        ]]
[[155.65134114  87.85717405   9.27784126   3.44601847   0.
    0.           0.           0.           0.           0.
    0.           0.        ]
 [155.51047505  87.41105057   8.86190178   3.359943     0.
    0.           0.           0.           0.           0.
    0.           0.        ]


[[4.59975400e+01 9.19103008e-01 1.07229759e-01 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [4.60890952e+01 8.74663139e-01 2.26826448e-02 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [4.60832663e+01 1.22976593e+00 6.10199500e-01 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [4.60933604e+01 8.41230067e-01 4.86188189e-02 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [4.59551229e+01 1.06972620e+00 4.09073154e-01 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]]
[[99.14674077 50.22446987  5.35458666  1.14005051  0.          0

[[1.38014979 0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [1.37432523 0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [1.37344131 0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [1.37422251 0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [1.38230299 0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]]
[[90.64233855  5.1014      1.34687353  0.          0.          0.
   0.          0.          0.          0.          0.          0.        ]
 [90.5233162   5.34574241  1.19199143  0.          0.          0.
   0.          0.          0.          0.          0.          0.        ]
 [90.48528194  4.73545082  1.46410144  0.          0.          0.
   0. 